# Supervised Fine-Tuning on CUDA

<a target="_blank" href="https://colab.research.google.com/github/simonguest/CS-394/blob/main/src/07/notebooks/train-cuda.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/simonguest/CS-394/raw/refs/heads/main/src/07/notebooks/train-cuda.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## Dependencies

In [1]:
!uv pip install -q datasets bitsandbytes trl

## Model parameters

In [2]:
import os

DATASET_REPO = "juvereturn/bible-dataset" # Change this to your uploaded dataset location on HF

BASE_MODEL_VENDOR = "Qwen" # Change this to your desired base model
BASE_MODEL_NAME = "Qwen3-0.6B" # Change this to your desired base model
BASE_MODEL = f"{BASE_MODEL_VENDOR}/{BASE_MODEL_NAME}"

PROJECT_NAME = "bible-assistant" # Name of your training project
os.environ["WANDB_PROJECT"] = PROJECT_NAME # WandB will use the same project name

MODEL_NAME = f"{BASE_MODEL_NAME}-{PROJECT_NAME}" # Final model name
HF_USERNAME = "juvereturn" # Hugging Face username - used to upload your final models

MODEL_FOLDER = f"./models/{MODEL_NAME}" # Local folder for storing the model files during training


## Training hyperparameters

In [3]:
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 2e-4
NUM_EPOCHS = 3
MAX_SEQ_LENGTH = 512

# LoRA-specific Parameters
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# Use 4-bit quantization for efficiency (QLoRA)
USE_4BIT = True

## API keys and tokens

In [4]:
import sys
import os
from dotenv import load_dotenv

if 'google.colab' in sys.modules:
  from google.colab import userdata # type:ignore
  os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
  os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')
  print("HF and WANDB API Tokens set for Colab")
else:
  load_dotenv()
  print("Loaded env vars from .env")

HF and WANDB API Tokens set for Colab


## Load dataset from Hugging Face

In [5]:
from datasets import load_dataset
dataset = load_dataset(DATASET_REPO)

README.md:   0%|          | 0.00/173 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/754k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/81.3k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/996 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/99 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

## Format dataset for correct chat template

In [6]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

def format_chat_template(example):
    messages = example["messages"]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

formatted_dataset = dataset.map(
    format_chat_template,
    remove_columns=dataset['train'].column_names # type: ignore
)

print("\nFormatted example:")
print(formatted_dataset['train'][0]['text'][:500])  # type: ignore

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Map:   0%|          | 0/996 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]


Formatted example:
<|im_start|>user
I've been hurt by someone who betrayed my trust, and I'm trying to forgive and move forward. I'm seeking guidance and peace rather than staying in anger.<|im_end|>
<|im_start|>assistant
<think>

</think>

{"issue_question": "I've been hurt by someone who betrayed my trust, and I'm trying to forgive and move forward. I'm seeking guidance and peace rather than staying in anger.", "verse": "Matthew 6:14-15 - For if ye forgive men their trespasses, your heavenly Father will forgive 


## Load base model with QLoRA configuration

In [7]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Configure 4-bit quantization
if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
else:
    bnb_config = None

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    dtype=torch.bfloat16,
)

# Prepare model for k-bit training
if USE_4BIT:
    model = prepare_model_for_kbit_training(model)

# Configure LoRA
peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

trainable params: 10,092,544 || all params: 761,724,928 || trainable%: 1.3250


## Create training configuration and trainer

In [8]:
import os
from transformers import TrainingArguments
from trl import SFTTrainer

report_to = "none"
#if os.environ.get("WANDB_API_KEY") != None:
  #report_to = "wandb"

# Training arguments
training_args = TrainingArguments(
    output_dir=MODEL_FOLDER,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    bf16=True,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    report_to=report_to,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["validation"],
    processing_class=tokenizer,
)

# Save the tokenizer model as this won't change during training
tokenizer.save_pretrained(f"{MODEL_FOLDER}/lora")

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Adding EOS to train dataset:   0%|          | 0/996 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/996 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/996 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

('./models/Qwen3-0.6B-bible-assistant/lora/tokenizer_config.json',
 './models/Qwen3-0.6B-bible-assistant/lora/chat_template.jinja',
 './models/Qwen3-0.6B-bible-assistant/lora/tokenizer.json')

## Train and save the final model

In [9]:
# Start training
trainer.train()

# Save the final model
trainer.save_model(f"{MODEL_FOLDER}/lora")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Step,Training Loss,Validation Loss
100,1.364535,1.402840
200,1.226186,1.247543
300,1.072740,1.198211
400,1.067908,1.150301
500,1.025651,1.115608
600,0.905480,1.117685
700,0.869999,1.116103


## Load the final model

In [10]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=f"{MODEL_FOLDER}/lora",
    tokenizer=f"{MODEL_FOLDER}/lora",
    temperature = 0.7,
    max_new_tokens = 256,
)

Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Loading weights:   0%|          | 0/392 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'temperature', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


## Test the final model using the test dataset

In [11]:
import random

idx = random.randint(0, len(formatted_dataset['test']) -1)
test_data = formatted_dataset['test'][idx]["text"]

output = pipe(test_data)
print(output[0]['generated_text'])

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<|im_start|>user
A seeker asks for forgiveness guidance.<|im_end|>
<|im_start|>assistant
<think>

</think>

{"issue_question": "I am hurting from a deep betrayal and I struggle to forgive. The pain keeps me from moving forward, and I fear bitterness will consume me. I want to forgive and heal, but I need clear steps and reassurance in this crisis moment.", "verse": "Matthew 6:14-15 - For if you forgive other people when they sin against you, your heavenly Father will also forgive you. But if you do not forgive others their sins, your Father will not forgive your sins.", "explanation": "These verses remind us that forgiveness is a path God invites us to walk, not a feeling we must manufacture alone. Forgiveness breaks the grip of bitterness, releasing you to receive God’s mercy and to extend mercy to others, even when the hurt remains real. It is not about denying the pain, but about choosing daily to release it to God and seek healing through His love. In crisis moments, this call is e

## Create a model card

In [12]:
from huggingface_hub import ModelCard

card_content = f"""---
base_model: {BASE_MODEL}
tags:
- peft
- lora
- text-generation
---

# {MODEL_NAME}

## Model Description
Fine-tuned from `{BASE_MODEL}` using QLoRA (4-bit) with supervised fine-tuning.

## Training Details
- Dataset: `{DATASET_REPO}`
- LoRA rank: {LORA_R}, alpha: {LORA_ALPHA}
- Epochs: {NUM_EPOCHS}, Learning rate: {LEARNING_RATE}

## Intended Use

This model is a test model used for the CS-394/594 class at DigiPen.

The model is a Bible assistant that can answer people's questions about the Bible and help them feel better. I believe the Bible has the ability to heal everyone in the world.

## Limitations

This model is a single-turn model and has not been trained on support long, multi-turn conversations.
"""

card = ModelCard(card_content)
card.save(f"{MODEL_FOLDER}/lora/README.md")

## Merge the models

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from peft import PeftModel

# Load the configuration and model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    dtype=torch.bfloat16,
)
adapter_model = PeftModel.from_pretrained(base_model, f"{MODEL_FOLDER}/lora")

# Merge and save the model
merged_model = adapter_model.merge_and_unload() # type: ignore
merged_model.save_pretrained(f"{MODEL_FOLDER}/merged")
# Save the tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.save_pretrained(f"{MODEL_FOLDER}/merged")

# Copy the model card to the merged folder
!cp {MODEL_FOLDER}/lora/README.md {MODEL_FOLDER}/merged/.

Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

## Upload the merged model and adapter to Hugging Face

In [14]:
from huggingface_hub import HfApi, create_repo

# Set the MODEL_REPO
MODEL_REPO = f"{HF_USERNAME}/{MODEL_NAME}"

# Initialize the API
api = HfApi()

# Create the repository (if it doesn't exist)
try:
    create_repo(MODEL_REPO, repo_type="model", exist_ok=True, private=False)
    print(f"Repository {MODEL_REPO} created or already exists")
except Exception as e:
    print(f"Error creating repo: {e}")

# Upload merged model files to root (this is the main model)
print("\nUploading merged model to root...")
api.upload_folder(
    folder_path=f"{MODEL_FOLDER}/merged",
    repo_id=MODEL_REPO,
    repo_type="model",
    path_in_repo="",  # Empty string uploads to root
    commit_message="Upload merged model"
)

# Upload LoRA adapter to subfolder (optional but useful for reference)
print("\nUploading LoRA adapter...")
api.upload_folder(
    folder_path=f"{MODEL_FOLDER}/lora",
    repo_id=MODEL_REPO,
    repo_type="model",
    path_in_repo="lora_adapter",  # Keep adapter in subfolder
    commit_message="Upload LoRA adapter"
)

print(f"\n✓ All files uploaded successfully to https://huggingface.co/{MODEL_REPO}")

Repository juvereturn/Qwen3-0.6B-bible-assistant created or already exists

Uploading merged model to root...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ant/merged/tokenizer.json:   0%|          | 28.3kB / 11.4MB            

  .../merged/model.safetensors:   3%|2         | 39.9MB / 1.50GB            

...

New Data Upload               : |          |  0.00B /  0.00B            

  ...stant/lora/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

  ...adapter_model.safetensors:  21%|##        | 8.45MB / 40.4MB            

  ...nt/lora/training_args.bin:  21%|##        | 1.18kB / 5.65kB            


✓ All files uploaded successfully to https://huggingface.co/juvereturn/Qwen3-0.6B-bible-assistant
